# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [42]:
# Import Python packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [43]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/samadarshad/dev/udacity/UdacityDataC2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [44]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [45]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [46]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [47]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS udacity
WITH REPLICATION =
{ 'class' : 'SimpleStrategy', 'replication_factor': 1 }
""")

#### Set Keyspace

In [48]:
session.set_keyspace('udacity')


### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [49]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
drop_table = """
DROP TABLE IF EXISTS session_library
"""
session.execute(drop_table)
create_table = """
CREATE TABLE IF NOT EXISTS session_library (sessionId int, itemInSession int, artist text, songTitle text, songLength decimal, PRIMARY KEY (sessionId, itemInSession))
"""
session.execute(create_table)
                    

##
ARTIST = 0
FIRST_NAME = 1
GENDER = 2
ITEM_IN_SESSION = 3
LAST_NAME = 4
LENGTH = 5
LEVEL = 6
LOCATION = 7
SESSION_ID = 8
SONG = 9
USER_ID = 10

In [50]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_library (sessionId, itemInSession, artist, songTitle, songLength) VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[SESSION_ID]),
                                int(line[ITEM_IN_SESSION]),
                                line[ARTIST],
                                line[SONG],
                                float(line[LENGTH]))
                        )

In [51]:
select_query = """
SELECT artist, songTitle, songLength FROM session_library WHERE sessionId = 338 AND itemInSession = 4
"""
rows = session.execute(select_query)
for r in rows.all():
    print(r)

Row(artist='Faithless', songtitle='Music Matters (Mark Knight Dub)', songlength=Decimal('495.3073'))


In [52]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182
drop_table = """
DROP TABLE IF EXISTS user_library
"""
session.execute(drop_table)
create_table = """
CREATE TABLE IF NOT EXISTS user_library (userId int, sessionId int, itemInSession int, artist text, songTitle text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))
"""
session.execute(create_table)


with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_library (userId, sessionId, itemInSession, artist, songTitle, firstName, lastName) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[USER_ID]),
                                int(line[SESSION_ID]),
                                int(line[ITEM_IN_SESSION]),
                                line[ARTIST],
                                line[SONG],
                                line[FIRST_NAME],
                                line[LAST_NAME],)
                        )

In [53]:
select_query = """
SELECT artist, songTitle, firstName, lastName FROM user_library WHERE userId = 10 AND sessionId = 182
"""
rows = session.execute(select_query)
for r in rows.all():
    print(r)

Row(artist='Down To The Bone', songtitle="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', songtitle='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', songtitle='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', songtitle='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [54]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
drop_table = """
DROP TABLE IF EXISTS song_library
"""
session.execute(drop_table)
create_table = """
CREATE TABLE IF NOT EXISTS song_library (songTitle text, userId int, firstName text, lastName text, PRIMARY KEY (songTitle, userId))
"""
session.execute(create_table)
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (songTitle, userId, firstName, lastName) VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[SONG],
                                int(line[USER_ID]),
                                line[FIRST_NAME],
                                line[LAST_NAME],)
                        )
                    

In [55]:
select_query = """
SELECT songTitle, firstName, lastName FROM song_library WHERE songTitle = 'All Hands Against His Own'
"""
rows = session.execute(select_query)
for r in rows.all():
    print(r)



Row(songtitle='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(songtitle='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(songtitle='All Hands Against His Own', firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [56]:
drop_table = """
DROP TABLE IF EXISTS session_library
"""
session.execute(drop_table)
drop_table = """
DROP TABLE IF EXISTS user_library
"""
session.execute(drop_table)
drop_table = """
DROP TABLE IF EXISTS song_library
"""
session.execute(drop_table)


### Close the session and cluster connection¶

In [57]:
session.shutdown()
cluster.shutdown()